In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

# Q1

In [7]:
!curl localhost:9200

{
  "name" : "a245605010fe",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "IOIof_fwQpKZWXrPeauzyQ",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


# Q2

In [8]:
from elasticsearch import Elasticsearch

In [9]:
es_client = Elasticsearch('http://localhost:9200')

In [10]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [11]:
index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [12]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████| 948/948 [00:20<00:00, 46.74it/s]


# Q3

# Explanation

## size
Specifies the number of results to return. In this case, it's set to 5, meaning the query will return up to 5 documents that match the search criteria.

## query
The top-level key indicating that the following section defines the query to execute.

## bool query
A boolean query allows combining multiple query clauses to achieve specific search goals.

### must clause
Contains a `multi_match` query, which searches for documents that match the `user_question` in multiple fields (`question`, `text`, `section`).
- The `query` parameter is dynamically filled with `user_question` (presumably a user-provided search query).
- `fields` specifies the fields to search in. The `^3` after `question` boosts its importance (weight) in the search results, indicating that matches in the `question` field are more significant.
- `type` is set to `best_fields`, which finds documents where any field matches and uses the highest score for that match.

### filter clause
Contains a `term` query that filters documents based on exact matching of the `course` field.
Specifically, it filters documents where the `course` field exactly matches "data-engineering-zoomcamp".

# Summary
This Elasticsearch query will return up to 5 documents where:
- The documents must match the `user_question` in at least one of the specified fields (`question`, `text`, `section`).
- The `question` field is given higher relevance (`^3`) in determining matches.
- The documents must also have the `course` field set to "data-engineering-zoomcamp".

This query structure is typical for searching specific fields with boosted relevance and applying filters to narrow down search results based on certain criteria.

In [22]:
def elastic_search_q3(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    print(response['hits']['hits'][0]['_score'])

In [23]:
query = "How do I execute a command in a running docker container?"

In [24]:
elastic_search_q3(query)

84.050095


# Q4

In [38]:
def elastic_search_q4(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    cnt = 3
    for hit in response['hits']['hits']:
        if (cnt > 0):
            result_docs.append(hit['_source']['question'])
            cnt -= 1;
        else:
            break;
    
    return result_docs

In [39]:
elastic_search_q4(query)

['How do I debug a docker container?',
 'How do I copy files from my local machine to docker container?',
 'How do I copy files from a different folder into docker container’s working directory?']

# Q5

In [44]:
def elastic_search_q5(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    cnt = 3
    for hit in response['hits']['hits']:
        if (cnt > 0):
            result_docs.append(hit['_source'])
            cnt -= 1;
        else:
            break;
    
    return result_docs

In [45]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [46]:
def build_prompt(query, search_results):
    context_template = """
    Q: {question}
    A: {text}
    """.strip()
    
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()
    context = ""
    
    for doc in search_results:
        context = context + context_template.format(question = doc['question'], text = doc['text']) + "\n\n"
    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [53]:
q5_response = build_prompt(query, elastic_search_q5(query))

In [54]:
len(q5_response)

1498

In [48]:
elastic_search_q5(query)

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

# Q6

In [49]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.6 MB/s eta 0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 17.8 MB/s eta 0:00:00m eta 0:00:01


In [51]:
import tiktoken

In [52]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [61]:
encoding_list = encoding.encode(text = q5_response)

In [59]:
len(encoding.encode(text = q5_response))

331

In [62]:
for encoded in encoding_list:
    print(encoding.decode_single_token_bytes(encoded))

b"You're"
b' a'
b' course'
b' teaching'
b' assistant'
b'.'
b' Answer'
b' the'
b' QUESTION'
b' based'
b' on'
b' the'
b' CONT'
b'EXT'
b' from'
b' the'
b' FAQ'
b' database'
b'.\n'
b'   '
b' Use'
b' only'
b' the'
b' facts'
b' from'
b' the'
b' CONT'
b'EXT'
b' when'
b' answering'
b' the'
b' QUESTION'
b'.\n'
b'    \n'
b'   '
b' QUESTION'
b':'
b' How'
b' do'
b' I'
b' execute'
b' a'
b' command'
b' in'
b' a'
b' running'
b' docker'
b' container'
b'?\n'
b'    \n'
b'   '
b' CONT'
b'EXT'
b':\n'
b'   '
b' Q'
b':'
b' How'
b' do'
b' I'
b' debug'
b' a'
b' docker'
b' container'
b'?\n'
b'   '
b' A'
b':'
b' Launch'
b' the'
b' container'
b' image'
b' in'
b' interactive'
b' mode'
b' and'
b' overriding'
b' the'
b' entry'
b'point'
b','
b' so'
b' that'
b' it'
b' starts'
b' a'
b' bash'
b' command'
b'.\n'
b'docker'
b' run'
b' -'
b'it'
b' --'
b'entry'
b'point'
b' bash'
b' <'
b'image'
b'>\n'
b'If'
b' the'
b' container'
b' is'
b' already'
b' running'
b','
b' execute'
b' a'
b' command'
b' in'
b' the'
b' specific'
b' 